In [49]:
import ee
#ee.Authenticate()
ee.Initialize()

In [2]:
#Import required packages
import geemap
import pandas as pd

Code source: [Ujaval Gandhi](https://twitter.com/spatialthoughts)

This example script shows how to calculate building density using the Global MS Buildings Footprints Dataset. This script uses Advanced Vector Processing techniques. To understand these concepts, read Chapter F5.3 of the open-access EEFA Book at https://www.eefabook.org/


### Here's the overview of the workflow
1. Create a regular grid of chosen size
2. Count the number of buildings in each grid
3. Color each grid polygon based on building density
4. Export the resulting map image.

## GEE Code

## Pythonic version - to be written

In [3]:
hyderabad_airshed = 'grids_hyderabad/grids_hyderabad.shp'

In [5]:
def get_geometry(airshed_shp):
    airshed_box = geemap.shp_to_ee(airshed_shp)
    geometry = airshed_box.geometry()
    return airshed_box, geometry

In [21]:
hyd_grids, geometry = get_geometry(hyderabad_airshed)

In [7]:
objects = ee.data.listAssets({'parent': 'projects/sat-io/open-datasets/MSBuildings'})

In [9]:
objects['assets']

[{'type': 'TABLE',
  'name': 'projects/earthengine-legacy/assets/projects/sat-io/open-datasets/MSBuildings/Abyei',
  'id': 'projects/sat-io/open-datasets/MSBuildings/Abyei',
  'updateTime': '2022-05-27T07:26:51.088326Z'},
 {'type': 'TABLE',
  'name': 'projects/earthengine-legacy/assets/projects/sat-io/open-datasets/MSBuildings/Afghanistan',
  'id': 'projects/sat-io/open-datasets/MSBuildings/Afghanistan',
  'updateTime': '2022-05-27T07:26:51.450428Z'},
 {'type': 'TABLE',
  'name': 'projects/earthengine-legacy/assets/projects/sat-io/open-datasets/MSBuildings/Africa',
  'id': 'projects/sat-io/open-datasets/MSBuildings/Africa',
  'updateTime': '2022-05-27T07:26:51.836817Z'},
 {'type': 'TABLE',
  'name': 'projects/earthengine-legacy/assets/projects/sat-io/open-datasets/MSBuildings/Albania',
  'id': 'projects/sat-io/open-datasets/MSBuildings/Albania',
  'updateTime': '2022-05-27T07:26:52.124487Z'},
 {'type': 'TABLE',
  'name': 'projects/earthengine-legacy/assets/projects/sat-io/open-datasets

In [10]:
buildingsCol = ee.FeatureCollection('projects/sat-io/open-datasets/MSBuildings/India')

In [14]:
buildings = buildingsCol.filter(ee.Filter.bounds(geometry))

In [33]:
intersectFilter = ee.Filter.intersects(leftField= '.geo', rightField='.geo', maxError= 10)

In [35]:
saveAllJoin = ee.Join.saveAll(matchesKey= 'buildings')

In [48]:
#gridSizeMeters = 1000
#grid = geometry.coveringGrid('EPSG:3857', gridSizeMeters)

In [37]:
joined = saveAllJoin.apply(hyd_grids, buildings, intersectFilter);

In [39]:
def count_buildings(image):
    buildingsWithin = ee.List(image.get('buildings'))
    totalBuildings = ee.FeatureCollection(buildingsWithin).size()
    return image.set('total_buildings', totalBuildings)

In [40]:
buildingCounts = joined.map(count_buildings)

In [43]:
task = ee.batch.Export.table.toDrive(**{
  'collection': buildingCounts.select(['total_buildings']),
  'description': 'Grid_With_Building_Count',
  'folder': 'earthengine',
  'fileNamePrefix': 'grid_with_count2',
  'fileFormat': 'SHP',
})

In [44]:
task.start()

In [46]:
# tasks = ee.batch.Task.list()
# for task in tasks:
#     task_id = task.status()['id']
#     task_state = task.status()['state']
#     print(task_id, task_state)